In [ ]:
#essentially, tool for making datasets from wikipedia. Ask for data around a particular topic and a for a brief analysis of it. Have the agent query wikipedia, format the results into a dataframe, and return a dataframe as well as some visualizataions

#search wikipedia for lists with tables
#parse tables into dataset
#display dataset and analyze


In [1]:
!nvidia-smi

Wed Sep  3 09:49:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:01:00.0 Off |                  N/A |
|  0%   41C    P8               6W / 170W |     11MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
[sudo] password for thoth: 


In [2]:
!pip install -U wikipedia langgraph-checkpoint-sqlite llama-cpp-python ollama llama-stack langchain_experimental

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 3.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [3]:
!pip install -U langchain-nomic langchain_community langchainhub chromadb langchain langgraph tavily-python nomic[local] langchain-text-splitters langchain-ollama #tiktoken

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached langchain_core-0.3.69-py3-none-any.whl.metadata (5.8 kB)
  Using cached sqlalchemy-2.0.41-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached langsmith-0.4.6-py3-none-any.whl.metadata (15 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached pydantic-2.11.7-py3-no

In [3]:
!ollama list

NAME                    ID              SIZE      MODIFIED      
qwen2.5-coder:latest    dae161e27b0e    4.7 GB    3 seconds ago    


In [2]:
!ollama pull qwen2.5-coder
#!ollama pull deepseek-r1:14b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 60e05f210007: 100% ▕██████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b: 100% ▕██████████████████▏   68 B                         
pulling 1e65450c3067: 100% ▕██████████████████▏ 1.6 KB                         
pulling 832dd9e00a68: 100% ▕██████████████████▏  11 KB                         
pulling d9bb33f27869: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
!ollama list

NAME                    ID              SIZE      MODIFIED       
qwen2.5-coder:latest    dae161e27b0e    4.7 GB    24 minutes ago    


In [4]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="qwen2.5-coder")

chain = prompt | model

chain.invoke({"question": "Who influenced Hegel?"})

'To answer the question "Who influenced Hegel?", we need to consider Hegel\'s intellectual heritage and the thinkers who shaped his philosophical ideas.\n\n1. **Georg Wilhelm Friedrich Hegel (1770-1831)** was a German philosopher known for his comprehensive system of philosophy, particularly in his work "Phenomenology of Spirit" and "Science of Logic".\n\n2. **Immanuel Kant (1724-1804)** is widely regarded as one of the most influential philosophers of modern times. Kant\'s critical philosophy addressed both metaphysics and epistemology and laid the groundwork for Hegel\'s own work.\n\n3. **Hegel was also influenced by Schelling**: Georg Wilhelm Friedrich Schelling (1775-1854) was a German idealist philosopher who contributed to early German romanticism and philosophy of nature. Schelling’s work in metaphysics, particularly his concept of the Absolute, had an impact on Hegel\'s own development.\n\n4. **Johann Gottlieb Fichte (1762-1814)** was another key influence. Fichte’s works, espe

In [6]:
#Importing the required packages
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
# from langchain_community.tools.openai_dalle_image_generation import (
#    OpenAIDALLEImageGenerationTool
# )
#from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
import os
import pandas as pd
from langchain_core.tools import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain.schema import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain_ollama import ChatOllama

from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


llm = ChatOllama(model="qwen2.5-coder", format="json", temperature=0)

memory = MemorySaver()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

@tool
def get_wiki_tables(url): #(url: string) -> list:
  """pull out the embedded html tables from a wikipedia page"""
  return pd.read_html(url)

tools = [wikipedia, multiply, get_wiki_tables]

agent_executor = create_react_agent(llm, tools)#, checkpointer=memory)

In [11]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

{"name": "wikipedia", "arguments": {"query": "Hello!"}}


In [9]:
llm_with_tools = llm.bind_tools([wikipedia, multiply])
response = llm_with_tools.invoke("what is 4 times 5?")
response.content

'{"name": "multiply", "arguments": {"a": 4, "b": 5}}'

In [10]:
response

AIMessage(content='{"name": "multiply", "arguments": {"a": 4, "b": 5}}', additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder', 'created_at': '2025-09-03T16:56:51.869954099Z', 'done': True, 'done_reason': 'stop', 'total_duration': 34566292254, 'load_duration': 40582954, 'prompt_eval_count': 261, 'prompt_eval_duration': 30304446589, 'eval_count': 22, 'eval_duration': 4215339238, 'model_name': 'qwen2.5-coder'}, id='run--5606926a-0338-47c2-a5f8-981e0bcbc606-0', usage_metadata={'input_tokens': 261, 'output_tokens': 22, 'total_tokens': 283})

In [12]:
@tool
def get_weather(location: str) -> str:
    """Get the weather from a location."""

    return "Sunny."


weather_llm_with_tools = llm.bind_tools([get_weather])

weather_response = weather_llm_with_tools.invoke("What's the weather in San Francisco, CA?")
weather_response.content

'{"name": "get_weather", "arguments": {"location": "San Francisco, CA"}}'

In [ ]:
weather_response

In [14]:
response = agent_executor.invoke({"messages": [HumanMessage(content="what is 4 times 5?")]})

In [ ]:
response['messages']

[HumanMessage(content='what is 4 times 5?', additional_kwargs={}, response_metadata={}, id='6db9581a-0896-4ae5-b22d-e5adda4ac2fd'),
 AIMessage(content='{"name": "multiply", "arguments": {"a": 4, "b": 5}}', additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder', 'created_at': '2025-06-20T22:05:49.756113151Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1003369238, 'load_duration': 80876263, 'prompt_eval_count': 178, 'prompt_eval_duration': 60149934, 'eval_count': 22, 'eval_duration': 836269737, 'model_name': 'qwen2.5-coder'}, id='run--1dd8361f-e279-41f8-afce-4a1323f7ac78-0', usage_metadata={'input_tokens': 178, 'output_tokens': 22, 'total_tokens': 200})]

In [15]:
response

{'messages': [HumanMessage(content='what is 4 times 5?', additional_kwargs={}, response_metadata={}, id='e9ec2083-d38a-4a07-97fd-8e03810af629'),
  AIMessage(content='{"name": "multiply", "arguments": {"a": 4, "b": 5}}', additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder', 'created_at': '2025-09-03T17:00:39.421730914Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19399862821, 'load_duration': 42790774, 'prompt_eval_count': 314, 'prompt_eval_duration': 14735423360, 'eval_count': 22, 'eval_duration': 4615655311, 'model_name': 'qwen2.5-coder'}, id='run--28139313-b7ab-4344-a5e7-ec35d27e0838-0', usage_metadata={'input_tokens': 314, 'output_tokens': 22, 'total_tokens': 336})]}

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Search wikipedia for the most densely populated cities in the world")]})

response["messages"]

[HumanMessage(content='Search wikipedia for the most densely populated cities in the world', additional_kwargs={}, response_metadata={}, id='0baba3c9-1bf3-4b69-9150-dc32c57a4370'),
 AIMessage(content='{\n  "name": "wikipedia",\n  "arguments": {\n    "query": "most densely populated cities in the world"\n  }\n}', additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder', 'created_at': '2025-06-20T22:41:06.736998369Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1102380232, 'load_duration': 23344272, 'prompt_eval_count': 264, 'prompt_eval_duration': 21949391, 'eval_count': 31, 'eval_duration': 1050075512, 'model_name': 'qwen2.5-coder'}, id='run--4432ded5-9182-4253-948c-937a3b873cf8-0', usage_metadata={'input_tokens': 264, 'output_tokens': 31, 'total_tokens': 295})]

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Can you tell me what the most densely populated cities in the world are?")]})

response["messages"]

[HumanMessage(content='Can you tell me what the most densely populated cities in the world are?', additional_kwargs={}, response_metadata={}, id='2a8dac9b-b216-4c55-98ea-1fbb3bfb7510'),
 AIMessage(content='{"name": "wikipedia", "arguments": {"query": "most densely populated cities in the world"}}', additional_kwargs={}, response_metadata={'model': 'qwen2.5-coder', 'created_at': '2025-06-20T22:26:48.761439258Z', 'done': True, 'done_reason': 'stop', 'total_duration': 28363515274, 'load_duration': 26972302996, 'prompt_eval_count': 268, 'prompt_eval_duration': 706295498, 'eval_count': 23, 'eval_duration': 662882092, 'model_name': 'qwen2.5-coder'}, id='run--8ec00883-8bad-4d88-a0aa-35cfb22ad18e-0', usage_metadata={'input_tokens': 268, 'output_tokens': 23, 'total_tokens': 291})]

In [ ]:
message_content = "Can you tell me what the most densely populated cities in the world are?"

config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=message_content)]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Can you tell me what the most densely populated cities in the world are?
================================== Ai Message ==================================

{"name": "wikipedia", "arguments": {"query": "most densely populated cities in the world"}}


In [ ]:
query = "Hi!"
response = llm.invoke([{"role": "user", "content": query}])
response.text()

'{\n    "name": "Qwen",\n    "description": "I am Qwen, an AI assistant developed by Alibaba Cloud. My purpose is to help users with a wide range of tasks and provide information on various topics."\n}'

In [ ]:

#if the topic asks for data, use the wikipedia tool
#if the topic asks for trends, use the chart tool

In [ ]:
#starting point: have agent that compiles wikipedia links

# Wikipedia tests

In [16]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.openai_dalle_image_generation import (
   OpenAIDALLEImageGenerationTool
)
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
import os
from langchain_core.tools import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain.schema import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain_ollama import ChatOllama

from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


#llm = ChatOllama(model="qwen2.5-coder", format="json", temperature=0)


#memory = MemorySaver()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(), load_all_available_meta=True)

#tools = [wikipedia]

#agent_executor = create_react_agent(llm, tools)#, checkpointer=memory)

In [5]:
wikipedia.invoke("I need some information on the most densely populated cities in the world")

"Page: Kowloon Walled City\nSummary: Kowloon Walled City (Chinese: 九龍寨城) was an extremely densely populated and largely lawless enclave of China within the boundaries of Kowloon City of former British Hong Kong. Built as an imperial Chinese military fort, the walled city became a de jure enclave after the New Territories were leased to the United Kingdom in 1898. Its population increased dramatically after the end of the Japanese occupation of Hong Kong during World War II, attracting mostly refugees fleeing the renewed Chinese Civil War.\nBy the late 1980s, the walled city contained roughly 35,000 residents within its territory of 2.6 hectares (6+1⁄2 acres); resulting in a staggering population density of over 1.2 million inhabitants per square kilometre (3 million inhabitants per square mile). As a result of the absence of any widely recognized bureaucracy, the city's residents and businesses had no municipal codes to govern them. Enabled by the enclave's anarchic nature, trade in ba

In [19]:
from typing import List, TypedDict#, Literal, Optional, Sequence, Annotated
from langchain_community.document_loaders import WikipediaLoader
query = "what are the oldest universities in the world?"

wikipedia_docs = WikipediaLoader(query, load_max_docs=2)

In [20]:
docs=wikipedia_docs.load()

top_url = docs[0].metadata['source']

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?prop=extracts%7Crevisions&explaintext=&rvprop=ids&titles=List+of+oldest+extant+buildings&format=json&action=query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7cceb2c8a7c0>: Failed to resolve 'en.wikipedia.org' ([Errno -3] Temporary failure in name resolution)"))

In [ ]:
tables_pd = get_wiki_tables.invoke(top_url)

In [ ]:
prompt_script = """
Write a python script that takes in a pandas dataframe and creates a {visualization}, where the dataframe is of the format:
{dataframe_info}
"""

llm.invoke(prompt_script.format(visualization="bar chart"))

In [ ]:
tables_pd[0].columns.get_level_values(1)

Index(['Year', 'University', 'Original', 'Current', 'Notes'], dtype='object')

In [ ]:
prompt_script = """
Write a python script that takes in a pandas dataframe and creates a {visualization}, where the dataframe is of the format:
{dataframe_info}
"""

print(prompt_script.format(visualization="bar chart", dataframe_info=output_string))


Write a python script that takes in a pandas dataframe and creates a bar chart, where the dataframe is of the format:
Column Data Types:
- Column '('Year', 'Year')': object
- Column '('University', 'University')': object
- Column '('Location', 'Original')': object
- Column '('Location', 'Current')': object
- Column '('Notes', 'Notes')': object




In [ ]:
df = tables_pd[0]
output_string = "Column Names and Data Types:\n"

for col_name, col_dtype in df.dtypes.items():
    output_string += f"- Column '{col_name}': {col_dtype}\n"

In [ ]:
final_prompt = prompt_script.format(visualization="bar chart", dataframe_info=output_string)
print(final_prompt)


Write a python script that takes in a pandas dataframe and creates a bar chart, where the dataframe is of the format:
Column Names and Data Types:
- Column '('Year', 'Year')': object
- Column '('University', 'University')': object
- Column '('Location', 'Original')': object
- Column '('Location', 'Current')': object
- Column '('Notes', 'Notes')': object




In [ ]:
res = llm.invoke(final_prompt)

In [ ]:
res

AIMessage(content='{ "thoughts": "To create a bar chart from the given pandas dataframe, we need to first ensure that the data is in a suitable format. Since the \'Year\' column is of type \'object\', we should convert it to an integer or float if possible. Then, we can group the data by \'University\' and calculate the count of each university for plotting. Finally, we will use matplotlib to create the bar chart.", "sql": "", "python": "import pandas as pd\\nimport matplotlib.pyplot as plt\\n\\n# Assuming df is your input dataframe\\ndf[\'Year\'] = pd.to_numeric(df[\'Year\'], errors=\'coerce\')  # Convert \'Year\' to numeric, coercing errors to NaN\\n\\n# Group by \'University\' and count the number of entries for each university\\ngrouped_df = df.groupby(\'University\').size().reset_index(name=\'Count\')\\n\\n# Create a bar chart\\nplt.figure(figsize=(10, 6))\\nplt.bar(grouped_df[\'University\'], grouped_df[\'Count\'], color=\'skyblue\')\\nplt.xlabel(\'University\')\\nplt.ylabel(\'Nu

In [ ]:
import json
print(json.loads(res.content)['python'])

import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your input dataframe
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')  # Convert 'Year' to numeric, coercing errors to NaN

# Group by 'University' and count the number of entries for each university
grouped_df = df.groupby('University').size().reset_index(name='Count')

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(grouped_df['University'], grouped_df['Count'], color='skyblue')
plt.xlabel('University')
plt.ylabel('Number of Entries')
plt.title('Bar Chart of University Entries by Year')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import json

json.loads(res.content)

{'thoughts': "To create a bar chart from the given pandas dataframe, we need to first ensure that the data is in a suitable format. Since the 'Year' column is of type 'object', we should convert it to an integer or float if possible. Then, we can group the data by 'University' and calculate the count of each university for plotting. Finally, we will use matplotlib to create the bar chart.",
 'sql': '',
 'python': "import pandas as pd\nimport matplotlib.pyplot as plt\n\n# Assuming df is your input dataframe\ndf['Year'] = pd.to_numeric(df['Year'], errors='coerce')  # Convert 'Year' to numeric, coercing errors to NaN\n\n# Group by 'University' and count the number of entries for each university\ngrouped_df = df.groupby('University').size().reset_index(name='Count')\n\n# Create a bar chart\nplt.figure(figsize=(10, 6))\nplt.bar(grouped_df['University'], grouped_df['Count'], color='skyblue')\nplt.xlabel('University')\nplt.ylabel('Number of Entries')\nplt.title('Bar Chart of University Entrie

In [ ]:
print(output_string)

Column Data Types:
- Column '('Year', 'Year')': object
- Column '('University', 'University')': object
- Column '('Location', 'Original')': object
- Column '('Location', 'Current')': object
- Column '('Notes', 'Notes')': object



In [ ]:
tables_pd[0].columns

MultiIndex([(      'Year',       'Year'),
            ('University', 'University'),
            (  'Location',   'Original'),
            (  'Location',    'Current'),
            (     'Notes',      'Notes')],
           )

In [ ]:
tables_pd[0]

Year  \
                                          Year   
0        1180–1190[18] (teaching from c. 1088)   
1        1200–1214[23] (teaching from c. 1096)   
2                                 1209–1225[5]   
3                                 1218–1219[5]   
4                                      1222[5]   
5                                      1224[5]   
6                                      1290[5]   
7             1293 (Papal recognition 1346)[5]   
8                                      1308[5]   
9                                      1348[5]   
10           1357[5] (originally 1246–1252)[5]   
11                                     1361[5]   
12                                     1364[5]   
13                                     1365[5]   
14                                     1385[5]   
15  c. 1400[5] (originally 1343 to c. 1360)[5]   
16                                     1404[5]   
17                                     1409[5]   
18                            1410[5]–1413[49]   
19                                     1419[5]   
20           1430[5] (originally 1391–1394)[5]   
21     1431[5] (originally 1303 to c. 1400)[5]   
22                                     1444[5]   
23                                     1450[5]   
24                                     1451[5]   
25                                     1456[5]   
26                                     1457[5]   
27                                     1459[5]   
28                            1459[5]–1472[67]   
29                                     1475[5]   
30                                     1476[5]   
31                                     1477[5]   
32                                     1495[5]   
33                                     1499[5]   
34                                     1500[5]   

                                University  \
                                University   
0                    University of Bologna   
1                     University of Oxford   
2                  University of Cambridge   
3                  University of Salamanca   
4                      University of Padua   
5         University of Naples Federico II   
6                    University of Coimbra   
7                 University of Valladolid   
8                    University of Perugia   
9                       Charles University   
10                     University of Siena   
11                     University of Pavia   
12                 Jagiellonian University   
13                    University of Vienna   
14  Ruprecht Karl University of Heidelberg   
15                      University of Pisa   
16                     University of Turin   
17                   University of Leipzig   
18               University of St. Andrews   
19                   University of Rostock   
20                   University of Ferrara   
21             Sapienza University of Rome   
22                   University of Catania   
23                 University of Barcelona   
24                   University of Glasgow   
25                University of Greifswald   
26   Albert Ludwigs University of Freiburg   
27                     University of Basel   
28  Ludwig Maximilian University of Munich   
29                University of Copenhagen   
30   Eberhard Karls University of Tübingen   
31                      Uppsala University   
32                  University of Aberdeen   
33        Complutense University of Madrid   
34                  University of Valencia   

                                      Location                               \
                                      Original                      Current   
0         Kingdom of Italy,  Holy Roman Empire               Bologna, Italy   
1                           Kingdom of England       Oxford, United Kingdom   
2                           Kingdom of England    Cambridge, United Kingdom   
3                              Kingdom of León             Salamanca, Spain   
4             

In [ ]:
tables_pd

[                                          Year  \
                                           Year   
 0        1180–1190[18] (teaching from c. 1088)   
 1        1200–1214[23] (teaching from c. 1096)   
 2                                 1209–1225[5]   
 3                                 1218–1219[5]   
 4                                      1222[5]   
 5                                      1224[5]   
 6                                      1290[5]   
 7             1293 (Papal recognition 1346)[5]   
 8                                      1308[5]   
 9                                      1348[5]   
 10           1357[5] (originally 1246–1252)[5]   
 11                                     1361[5]   
 12                                     1364[5]   
 13                                     1365[5]   
 14                                     1385[5]   
 15  c. 1400[5] (originally 1343 to c. 1360)[5]   
 16                                     1404[5]   
 17                            

In [ ]:
query = "what are the oldest universities in the world?"
docs = wikipedia_docs.invoke(query, load_max_docs=2)
docs[0].metadata

In [ ]:
res1 = wikipedia.invoke("what are the oldest universities in the world?")

In [ ]:
wikipedia.invoke("what are the oldest universities in the world?")

'Page: List of oldest universities in continuous operation\nSummary: This is a list of the oldest existing universities in continuous operation in the world.\nInclusion in this list is determined by the date at which the educational institute first met the traditional definition of a university used by academic historians although it may have existed as a different kind of institution before that time. This definition limits the term "university" to institutions with distinctive structural and legal features that developed in Europe, and which make the university form different from other institutions of higher learning in the pre-modern world, even though these may sometimes now be referred to popularly as universities.\nTo be included in the list, the university must have been founded prior to 1500 in Europe or be the oldest university derived from the medieval European model in a country or region. It must also still be in operation, with institutional continuity retained throughout

In [ ]:
wikipedia.invoke("I need some information on the most densely populated cities in the world")

"Page: Kowloon Walled City\nSummary: Kowloon Walled City (Chinese: 九龍寨城) was an extremely densely populated and largely lawless enclave of China within the boundaries of Kowloon City of former British Hong Kong. Built as an imperial Chinese military fort, the walled city became a de jure enclave after the New Territories were leased to the United Kingdom in 1898. Its population increased dramatically after the end of the Japanese occupation of Hong Kong during World War II, attracting mostly refugees fleeing the renewed Chinese Civil War.\nBy the late 1980s, the walled city contained roughly 35,000 residents within its territory of 2.6 hectares (6.4 acres; 26,000 m2), or approximately four football fields; resulting in a staggering population density of over 3 million people per square mile. As a result of the absence of any widely recognized bureaucracy, the city's residents and businesses had no municipal codes to govern them. Enabled by the enclave's anarchic nature, trade in banned

In [ ]:
wikipedia.invoke("most densely populated cities in the world")

'Page: List of cities proper by population density\nSummary: This is a list of cities worldwide by population density. The population, population density and land area for the cities listed are based on the entire city proper, the defined boundary or border of a city or the city limits of the city. The population density of the cities listed is based on the average number of people living per square kilometer or per square mile. This list does not refer to the population, population density or land area of the greater metropolitan area or urban area, nor particular districts in any of the cities listed.\n\n\n\nPage: Bnei Brak\nSummary: Bnei Brak (Hebrew: בני ברק ) or Bene Beraq, is a city located on the central Mediterranean coastal plain in Israel, just east of Tel Aviv. A center of Haredi Judaism, Bnei Brak covers an area of 709 hectares (1,752 acres, or 2.74 square miles), and had a population of 218,357 in 2022. It is one of the most densely populated cities in Israel and the fourt

In [ ]:
!pip -q install langchain openai tiktoken

In [15]:
!pip -q install wikipedia duckduckgo-search langchain_experimental

    PyYAML (>=5.1.*)
            ~~~~~~^


In [16]:
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
import os
from langchain_core.tools import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain.schema import SystemMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain_ollama import ChatOllama

from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


llm = ChatOllama(model="qwen2.5-coder", format="json", temperature=0)
wikipedia = WikipediaAPIWrapper()
python_repl = PythonREPL()
search = DuckDuckGoSearchRun()

from langchain.agents import Tool

tools = [
    Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    ),
    Tool(
        name='wikipedia',
        func= wikipedia.run,
        description="Useful for when you need to look up a topic, country or person on wikipedia"
    ),
    Tool(
        name='DuckDuckGo Search',
        func= search.run,
        description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
    )
]


In [17]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [18]:
zero_shot_agent.invoke("When was Socrates born?")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `{ "thought": "To find out when Socrates was born, I should look up information about him on Wikipedia." }`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [ ]:
zero_shot_agent.run('Is 11 a prime number?')



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `{ "thought": "To determine if 11 is a prime number, I need to check if it has any divisors other than 1 and itself. A prime number is only divisible by 1 and itself." }`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [ ]:

def choose_visualization(self, state: dict) -> dict:
    """Choose an appropriate visualization for the data."""
    question = state['question']
    results = state['results']
    sql_query = state['sql_query']

    if results == "NOT_RELEVANT":
        return {"visualization": "none", "visualization_reasoning": "No visualization needed for irrelevant questions."}

    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
You are an AI assistant that recommends appropriate data visualizations. Based on the user's question, SQL query, and query results, suggest the most suitable type of graph or chart to visualize the data. If no visualization is appropriate, indicate that.

Available chart types and their use cases:
- Bar Graphs: Best for comparing categorical data or showing changes over time when categories are discrete and the number of categories is more than 2. Use for questions like "What are the sales figures for each product?" or "How does the population of cities compare? or "What percentage of each city is male?"
- Horizontal Bar Graphs: Best for comparing categorical data or showing changes over time when the number of categories is small or the disparity between categories is large. Use for questions like "Show the revenue of A and B?" or "How does the population of 2 cities compare?" or "How many men and women got promoted?" or "What percentage of men and what percentage of women got promoted?" when the disparity between categories is large.
- Scatter Plots: Useful for identifying relationships or correlations between two numerical variables or plotting distributions of data. Best used when both x axis and y axis are continuous. Use for questions like "Plot a distribution of the fares (where the x axis is the fare and the y axis is the count of people who paid that fare)" or "Is there a relationship between advertising spend and sales?" or "How do height and weight correlate in the dataset? Do not use it for questions that do not have a continuous x axis."
- Pie Charts: Ideal for showing proportions or percentages within a whole. Use for questions like "What is the market share distribution among different companies?" or "What percentage of the total revenue comes from each product?"
- Line Graphs: Best for showing trends and distributionsover time. Best used when both x axis and y axis are continuous. Used for questions like "How have website visits changed over the year?" or "What is the trend in temperature over the past decade?". Do not use it for questions that do not have a continuous x axis or a time based x axis.

Consider these types of questions when recommending a visualization:
1. Aggregations and Summarizations (e.g., "What is the average revenue by month?" - Line Graph)
2. Comparisons (e.g., "Compare the sales figures of Product A and Product B over the last year." - Line or Column Graph)
3. Plotting Distributions (e.g., "Plot a distribution of the age of users" - Scatter Plot)
4. Trends Over Time (e.g., "What is the trend in the number of active users over the past year?" - Line Graph)
5. Proportions (e.g., "What is the market share of the products?" - Pie Chart)
6. Correlations (e.g., "Is there a correlation between marketing spend and revenue?" - Scatter Plot)

Provide your response in the following format:
Recommended Visualization: [Chart type or "None"]. ONLY use the following names: bar, horizontal_bar, line, pie, scatter, none
Reason: [Brief explanation for your recommendation]
'''),
        ("human", '''
User question: {question}
SQL query: {sql_query}
Query results: {results}

Recommend a visualization:'''),
    ])

    response = llm.invoke(prompt, question=question, sql_query=sql_query, results=results)

    lines = response.split('\n')
    visualization = lines[0].split(': ')[1]
    reason = lines[1].split(': ')[1]

    return {"visualization": visualization, "visualization_reason": reason}

In [ ]:
def create_python_visualization(self, state: dict) -> dict:
    """Create a python visualization for the data."""
    question = state['question']
    results = state['results']
    sql_query = state['sql_query']

In [ ]:
#
tables_df = tables_tool.invoke("")
#choose_visualization
#visualize_data

In [ ]:
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)